## Import's

In [ ]:
import tensorflow as tf
from datasets import Dataset
from transformers import AutoTokenizer
from transformers import TFAutoModelForSequenceClassification
from transformers import create_optimizer
from keras import backend as K
import numpy as np
import json
import os

## Constantes

In [ ]:
## Constantes de diretório
DIR = './' ## Diretório 
dataset_path = "data/" ## Diretório raiz dos dataset's

## Constantes do modelo
model_id = 'neuralmind/bert-base-portuguese-cased'
epochs = 3
batchs = 16
qtd_classes = 3

## Tokenizador

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [ ]:
def tokenize_dataset(data):
    return tokenizer(data["text"], padding=True, return_tensors="tf")

## Modelo

In [ ]:
model = TFAutoModelForSequenceClassification.from_pretrained(model_id, num_labels=qtd_classes)

## Lendo datasets

In [ ]:
dict_datasets : dict[str, Dataset] = dict({})
for arquivo_nome in list(filter(lambda x: x.endswith(".json"), os.listdir(DIR+dataset_path))):
    dict_datasets.update(dict({arquivo_nome: Dataset.from_json(DIR+dataset_path+arquivo_nome)}))

In [ ]:
dict_datasets